In [ ]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular, train_log, train_super_batch, train_super_batch_KL, train_super_batch_L1, train_regular_L1,train_regular_one_hot
from model_saver import model_saver, model_saver_wandb
import inspect

In [ ]:
torch.tensor(0).to('cuda:0')

tensor(0, device='cuda:0')

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [ ]:
torch.set_default_tensor_type(torch.FloatTensor)

In [ ]:
torch.__version__

'1.12.1'

In [ ]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [ ]:
def compare_model_to_bf(model:GRUNet,raceDB:Races,example_ct):
    with torch.no_grad():
        sft_max = nn.Softmax(dim=-1)
        l_sftmax = nn.LogSoftmax(dim=-1)
        nnl_loss = nn.KLDivLoss(reduction='batchmean')
        full_test_races = raceDB.get_test_input(range(0,len(raceDB.test_race_ids)))
        full_test_races_w_prices = []
        excluded, included = 0,0
        for r in full_test_races:
            if 0 in r.prices or -1 in r.prices:
                excluded+=1
            else:
                full_test_races_w_prices.append(r)
                included+=1
        print(included,excluded)

        output = l_sftmax(model(full_test_races_w_prices))
        bf_prices = torch.log(torch.tensor([x.implied_prob for x in full_test_races_w_prices ]).to('cuda:0'))
        full_classes = torch.stack([x.classes for x in full_test_races_w_prices ])

        print()

        print(f"our loss = {nnl_loss(output,full_classes)}")
        print(f"their loss = {nnl_loss(bf_prices ,full_classes)}")
        wandb.log({"our loss":nnl_loss(output,full_classes), "their loss":nnl_loss(bf_prices ,full_classes)})

In [ ]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
#dog_stats_file = open( 'new gru input 2023-01.npy', 'rb')
hidden_size = 64
raceDB = build_dataset('gru_inputs_new_pir.npy', hidden_size ,state_filter="NSW", margin_type='boosted_sftmin')

(1168053, 19)
(1168053, 19)
(349251, 22)
Latest date = 2023-03-06 00:00:00
num_features_per_dog=77


  0%|          | 0/14800 [00:00<?, ?it/s]c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
  0%|          | 0/48862 [00:00<?, ?it/s]c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
100%|██████████| 48862/48862 [01:31<00:00, 532.08it/s]


number of races = 48862, number of unique dogs = 14800


In [ ]:
raceDB.create_new_weights_v2()
#

0        (Healesville, 350.0)
1            (Horsham, 410.0)
2        (The Meadows, 525.0)
3        (Warrnambool, 390.0)
4        (The Meadows, 525.0)
                 ...         
48857       (Ballarat, 390.0)
48858       (Ballarat, 390.0)
48859       (Ballarat, 450.0)
48860       (Ballarat, 450.0)
48861       (Ballarat, 545.0)
Length: 48862, dtype: object


In [ ]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 39923, Test examples 8939


In [ ]:
input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0]
model = smalll_prelin_GRUNet(input_size,64)
prev_model_file = 'polar-paper-2'

if prev_model_file!=None:
    model_name = prev_model_file
    model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
    model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
    raceDB.fill_hidden_states_from_dict(hidden_dict=model_data['db'])
    model.load_state_dict(model_data['model_state_dict'])
    

In [ ]:
#Testing
def predict_model(model,predDB):
    torch.autograd.set_detect_anomaly(True)
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():

        races_idx = range(0,len(predDB.raceIDs)-1)
        race = predDB.get_race_input(races_idx)
        X = race
        # for i,r in enumerate(race):
        #     print(r.raceid, r.track_name)
        #     #print(i,r.lstm_input())

        output = model(X)
        
        print(output)

        _, predicted = torch.max(output.data, 1)

        for i,r in enumerate(race):
            print(r.raceid, r.track_name, r.dogs[predicted[i].item()])

        print(predicted)



In [ ]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None,prev_model_file=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="GRU - FastTrack - AUS Testing - v2", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy

      model = smalll_prelin_GRUNet(input_size,config['hidden_size'])
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        raceDB.fill_hidden_states_from_dict(hidden_dict=model_data['db'])
        model.load_state_dict(model_data['model_state_dict'])
        config['parent model'] = prev_model_file


      raceDB.to_cuda()

      criterion = nn.CrossEntropyLoss(reduction='none')
      #criterion = nn.SmoothL1Loss(reduction='none', beta=0.1)
      # optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
      # optimizer = optim.Adadelta(model.parameters())
      optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])
      # optimizer = optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9)
      # optimizer = optim.ASGD(model.parameters(), lr=config['learning_rate'])

      print(criterion, optimizer)

      scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',threshold=0.0001, patience=10000, verbose=True, factor=0.5)
      model = model.to(device)
      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        train_regular(model, dataset, criterion, optimizer, scheduler, config)
      except KeyboardInterrupt:
        print("finished Early")
      dataset.create_hidden_states_dict()
      model_saver_wandb(model, optimizer, 450, 0.1, dataset.hidden_states_dict_gru, model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [38]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'batch_size': 500, 'dropout': 0.3, 'epochs': 2000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.000087, 'loss': 'L1', 'l1_beta':0.1,  'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1,'batch_before_backwards':7}

In [39]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='polar-paper-2')

{'hidden_size': 64, 'batch_size': 500, 'dropout': 0.3, 'epochs': 2000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 8.7e-05, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}
2000
{'hidden_size': 64, 'batch_size': 500, 'dropout': 0.3, 'epochs': 2000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 8.7e-05, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}


100%|██████████| 48862/48862 [00:03<00:00, 15293.34it/s]


filled =1901
empty  =388995
0.004863186116000164null_dog=0
CrossEntropyLoss() RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    foreach: None
    lr: 8.7e-05
    momentum: 0
    weight_decay: 0
)
smalll_prelin_GRUNet(
  (batchnorm): BatchNorm1d(686, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gru1): GRUCell(686, 64)
  (gru2): GRUCell(686, 64)
  (gru3): GRUCell(686, 64)
  (gru4): GRUCell(686, 64)
  (gru5): GRUCell(686, 64)
  (gru6): GRUCell(686, 64)
  (gru7): GRUCell(686, 64)
  (gru8): GRUCell(686, 64)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=512, out_features=64, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=64, out_features=8, bias=True)
  (output_fn): Identity()
)


  4%|▍         | 86/2000 [3:31:55<78:36:40, 147.86s/it]


finished Early
created path



FK ROI,█▁▂▁▂▂▂▂▁▂▁▂▂▂▃▃▃▃▃▃▃▄▅▃▄▄▄▄▃▄▅▄▃▃▄▄▄▃▂▄
FK ROI < 30,█▁▂▁▃▂▂▃▃▃▂▃▃▄▅▅▃▃▃▃▃▄▇▄▄▅▅▇▆▆▆▇▆▅▆▆▄▃▃▄
ROI,█▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▆▄▄▄▄▄▃▄▅▄▃▃▄▃▄▃▂▄
ROI < 30,█▁▁▁▃▃▃▃▄▄▄▄▄▅▆▆▅▅▅▅▅▆█▆▅▆▆▇▆▆▆▇▆▆▆▅▅▄▄▄
accuracy,▂▁▁▂▂▂▂▃▄▃▄▄▄▄▄▄▄▄▅▄▅▅▅▄▆▅▆▆▆▆▆▇▇▇▇▇▇▇█▇
batch_before_backwards,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_loss,█████▁█▁█▁▇█▇▇▇▇▁▇▁▇▁▇▇▇▇▁▇▁▇▁▇▁▇▇▇▇▁▇▁▇
correct,▂▁▁▂▂▂▂▃▄▃▄▄▄▄▄▄▄▄▅▄▅▅▅▄▆▅▆▆▆▆▆▇▇▇▇▇▇▇█▇
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_loss,█▇▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▃▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▁▂▁▁▁▁
loss_val,█▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁


In [12]:
sweep_config = {"method": "bayes"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "hidden_size": {"value":64}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [50]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate":{
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [100,250,500,1000,5000]
        },
        "batch_before_backwards": {
            'values': [5,10,20]
        },

        # "batch_before_backwards": {
        #     # a flat distribution between 0 and 0.1
        #     "distribution": "uniform",
        #     "min": 3,
        #     "max": 50,
        # },
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'ROI < 30'},
 'parameters': {'batch_before_backwards': {'values': [5, 10, 20]},
                'batch_size': {'values': [100, 250, 500, 1000, 5000]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [50]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'value': 64},
                'l1_beta': {'value': 0.1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.001,
                                  'min': 1e-05},
                'len_data': {'value': 40482},
                'loss': {'values': ['CEL']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'bayes',
 'metric': {'name': 'ROI < 30', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'len_data': {'value': 40482},
  'hidden_size': {'value': 64},
  'epochs': {'values': [50]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['CEL']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 1e-05, 'max': 0.001},
  'l1_beta': {'value': 0.1},
  'batch_size': {'values': [100, 250, 500, 1000, 5000]},
  'batch_before_backwards': {'values': [5, 10, 20]}}}

In [13]:
sweep_id = wandb.sweep(sweep_config, project="GRU_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: vjcqqbaf
Sweep URL: https://wandb.ai/nickojelly/GRU_sweeps/sweeps/vjcqqbaf


wandb: Agent Starting Run: n9de0evp with config:
wandb: 	batch_before_backwards: 20
wandb: 	batch_size: 1000
wandb: 	dropout: 0.3
wandb: 	epochs: 50
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 64
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.0008968546486164413
wandb: 	len_data: 40482
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 20, 'batch_size': 1000, 'dropout': 0.3, 'epochs': 50, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0008968546486164413, 'len_data': 40482, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
50
{'batch_before_backwards': 20, 'batch_size': 1000, 'dropout': 0.3, 'epochs': 50, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.0008968546486164413, 'len_data': 40482, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
CrossEntropyLoss() RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    foreach: None
    lr: 0.0008968546486164413
    momentum: 0
    weight_decay: 0
)
smalll_prelin_GRUNet(
  (batchnorm): BatchNorm1d(510, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gru1): GRUCell(510, 64)
  (gru2): GRUCell(510, 64)
  (gru3): GRUCell(510, 64)
  (gru4): GRUCell(51

100%|██████████| 50/50 [1:17:20<00:00, 92.82s/it]


...

created path



FK ROI,▁▂▃▃▄▄▅▅▆▆▆▇▆▆▇▇▆▆▆▇▇▆▇▇▇██████▇▇▇▇▇▇▇██
FK ROI < 30,▆▆▇▇█▇███▇▇▇▇▇▇▇▆▅▅▅▅▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▁▁▁▂
ROI,▁▂▃▄▅▅▅▅▆▆▆▇▆▆▇▇▆▆▆▇▇▆▆▆▇▇█▇▇▇▇▇▇▇▆▆▆▇▇▇
ROI < 30,▅▅▆▇█▇███▇▇▇▇▇▇▇▆▅▅▅▅▄▄▃▃▃▃▂▃▂▂▂▂▂▂▁▁▁▁▁
accuracy,▁▁▂▂▃▃▄▄▅▅▅▅▅▆▆▅▆▆▆▆▆▆▆▇▇▇▇████▇████████
batch_before_backwards,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_loss,███▂▇▇▂▆▇▇▆▇▇▇▇▇▇▇▇▁▇▇▁▆▇▁▆▆▆▆▆▆▆▆▆▆▆▆▁▆
correct,▁▁▂▂▃▃▄▄▅▅▅▅▅▆▆▅▆▆▆▆▆▆▆▇▇▇▇████▇████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch_loss,██▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▁▁
loss_val,█▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: bkajow1z with config:
wandb: 	batch_before_backwards: 5
wandb: 	batch_size: 500
wandb: 	dropout: 0.3
wandb: 	epochs: 50
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 64
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.000439383053806039
wandb: 	len_data: 40482
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 5, 'batch_size': 500, 'dropout': 0.3, 'epochs': 50, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.000439383053806039, 'len_data': 40482, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
50
{'batch_before_backwards': 5, 'batch_size': 500, 'dropout': 0.3, 'epochs': 50, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 64, 'l1_beta': 0.1, 'learning_rate': 0.000439383053806039, 'len_data': 40482, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
CrossEntropyLoss() RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    foreach: None
    lr: 0.000439383053806039
    momentum: 0
    weight_decay: 0
)
smalll_prelin_GRUNet(
  (batchnorm): BatchNorm1d(510, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gru1): GRUCell(510, 64)
  (gru2): GRUCell(510, 64)
  (gru3): GRUCell(510, 64)
  (gru4): GRUCell(510, 64)


 50%|█████     | 25/50 [4:26:29<4:55:05, 708.21s/it]